# Estudio índices climáticos (Mildiu Vid)
## Pablo Lavín
## TFM - Master Ciencia de Datos (UC-UIMP)
## Beca JAE Intro ICU 2025
## Septiembre 2025 - Junio 2026

En este notebook se realiza una intercomparación entre las observaciones de PTI-grid-v0 y del reanálisis ERA5-Land durante el periodo 1981–2021.

Los datos de temperatura se convierten en índices, en este caso, un índice compuesto de temperatura mínima (número de días por debajo una temperatura umbral) y de precipitación, con el objetivo de estudiar el riesgo de aparición de mildiu, un tipo de epidemia que afecta a la vid, y es crítica durante los meses de floración (mayo a junio).

Se incluyen métricas de sesgo, correlación, RMSE y ratio de varianzas.

## Configuración

In [1]:
# Cargamos paquetes
source("../../../scripts/setup_libraries.R")
source("../../../scripts/load_bc_functions.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


Registered S3 method overwritten by 'verification':
  method    from
  lines.roc pROC

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading requi

In [2]:
# Region de estudio

lon = c(-10, 5)
lat = c(35,44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

## Cargo datos ERA5-Land

In [3]:
for (m in 1:12) {
    # Temperatura mínima
    assign(paste0("era5_tmin_", sprintf("%02d", m)),
           readRDS(paste0("../../../data/compound_study_index/high_resolution/era5_land_data/era5_tmin_", sprintf("%02d", m), ".rds")))

    # Precipitación
    assign(paste0("era5_pr_", sprintf("%02d", m)),
           readRDS(paste0("../../../data/compound_study_index/high_resolution/era5_land_data/era5_pr_", sprintf("%02d", m), ".rds")))
}

## Cargo datos PTI

In [4]:
for (m in 1:12) {
    # Temperatura mínima
    assign(paste0("pti_tmin_", sprintf("%02d", m)),
           readRDS(paste0("../../../data/compound_study_index/high_resolution/pti_v0_data/pti_tmin_", sprintf("%02d", m), ".rds")))

    # Precipitación
    assign(paste0("pti_pr_", sprintf("%02d", m)),
           readRDS(paste0("../../../data/compound_study_index/high_resolution/pti_v0_data/pti_pr_", sprintf("%02d", m), ".rds")))
}

## Máscara para los datos

In [7]:
## Calculo el número de días que tmax > 22 grados (solo para la estructura del grid)
nd_obs = indexGrid(tx = era5_tmin_01, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(era5_tmin_01),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Funciones auxiliares

In [8]:
# Calcula la media anual de días que cumplen una condición conjunta de temperatura
# y humedad relativa a partir de datos diarios.
#
# Esta función se utilizado para datos de observaciones con dimeniones [time, lat, lon]
#
# @param tas_obs Lista con los datos diarios de temperatura y su información asociada.
# @param pr_obs Lista con los datos diarios de precipitación y su información asociada.
# @param temp_thresh Umbral de temperatura (por defecto 10 ºC).
# @param pr_thresh Umbral de precipitación (por defecto 10 mm).
#
# @return Lista tipo "grid" con la media anual de días que cumplen la condición.
#

compute_masked_obs = function(tas_obs, pr_obs, temp_thresh = 10, pr_thresh = 10) {
    
    # Vector de fechas diarias
    dates = as.Date(tas_obs$Dates$start)
    years = factor(format(dates, "%Y"))
    unique_years = levels(years)
    n_years = length(unique_years)
    nlat = dim(tas_obs$Data)[2]
    nlon = dim(tas_obs$Data)[3]
    
    # Condición conjunta: tas > temp_thresh y pr > pr_thresh
    mask = (tas_obs$Data > temp_thresh) & (pr_obs$Data >= pr_thresh)
    
    # Array [time(year), lat, lon]
    annual_days = array(NA, dim = c(n_years, nlat, nlon))
    
    for (y in 1:n_years) {
        idx = which(years == unique_years[y])
        annual_days[y,,] = apply(mask[idx,,], c(2,3), sum, na.rm = TRUE)
    }
    
    # Reconstruyo el grid
    grid = list()
    grid$Data = annual_days
    attr(grid$Data, "dimensions") = c("time", "lat", "lon")
    grid$xyCoords = tas_obs$xyCoords
    grid$Variable = tas_obs$Variable
    grid$Dates = tas_obs$Dates
    class(grid) = "grid"
    
    # Aplico la máscara de los datos
    grid = gridArithmetics(grid, mask.bin.spain, operator = "*")
    
    return(grid)
}

## Valores medio de las variables por mes

### ERA5-Land

In [ ]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("era5_tmin_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("tmin_era5_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(-10, 25, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("tmin_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("tmin_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Temperatura mínima media (1981-2021) (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

In [ ]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("era5_pr_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("pr_era5_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 15, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("pr_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("pr_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Precipitación media (1981-2021) (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

### PTI-grid-v0

In [ ]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("pti_tmin_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("tmin_pti_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(-10, 25, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("tmin_pti_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("tmin_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Temperatura mínima media (1981-2021) (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

In [ ]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    var = get(paste0("pti_pr_", mes))
    
    # Representación y guardado con sufijo
    assign(paste0("pr_pti_", mes), spatialPlot(
        climatology(var), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 15, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

png("pr_pti_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("pr_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Precipitación media (1981-2021) (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

## Riesgo aparición mildiu (Nº días con Tmin > 10ºC y PR > 10)

### ERA5-Land

In [12]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    tmin_var = get(paste0("era5_tmin_", mes))
    pr_var   = get(paste0("era5_pr_", mes))
  
    # Aplico la máscara y la guardo con el sufijo
    assign(paste0("grid_masked_", mes),
           compute_masked_obs(tmin_var, pr_var,
                              temp_thresh = 10,
                              pr_thresh   = 10))
  
    # Recupero el objeto recién creado
    grid_masked_obs = get(paste0("grid_masked_", mes))
    assign(paste0("grid_masked_era5_", mes), grid_masked_obs)
    
    # Representación y guardado con sufijo
    assign(paste0("nd_obs_era5_", mes), spatialPlot(
        climatology(grid_masked_obs), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 31, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [8]:
png("ndays_riesgo_mildiu_era5Land_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("nd_obs_era5_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Número de días de riesgo de mildiu (ERA5-Land)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

### PTI-grid-v0

In [13]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
  
    # Construyo los nombres de las variables de entrada
    tmin_var = get(paste0("pti_tmin_", mes))
    pr_var   = get(paste0("pti_pr_", mes))
  
    # Aplico la máscara y la guardo con el sufijo
    assign(paste0("grid_masked_", mes),
           compute_masked_obs(tmin_var, pr_var,
                              temp_thresh = 10,
                              pr_thresh   = 10))
  
    # Recupero el objeto recién creado
    grid_masked_obs = get(paste0("grid_masked_", mes))
    assign(paste0("grid_masked_pti_", mes), grid_masked_obs)
    
    # Representación y guardado con sufijo
    assign(paste0("nd_obs_pti_", mes), spatialPlot(
        climatology(grid_masked_obs), backdrop.theme = "countries",
        main = paste("Mes", mes), col.regions = color, at = seq(0, 31, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [10]:
png("ndays_riesgo_mildiu_ptiv0_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots 
plots = mget(paste0("nd_obs_pti_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(grobs = plots, ncol = 4,
             top   = textGrob("Número de días de riesgo de mildiu (PTI-grid-v0)",
                              gp = gpar(fontsize = 16, fontface = "bold")))

dev.off()

pdf 
  2

## Bias ndays

In [27]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
    
    # Calcular diferencias
    assign(paste0("diff_", mes),
           gridArithmetics(climatology(get(paste0("grid_masked_era5_", mes))),
                           climatology(get(paste0("grid_masked_pti_", mes))),
                           operator = "-") %>% suppressMessages %>% suppressWarnings)
    
    # Graficar
    assign(paste0("b_", mes),
           spatialPlot(get(paste0("diff_", mes)),
                       backdrop.theme = "countries",
                       main = paste("Bias ndays (Mes", mes, ")"),
                       col.regions = color,
                       at = seq(-5, 5, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [28]:
png("bias_ndays_mildiu_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots
plots = mget(paste0("b_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(
    grobs = plots,
    ncol = 4,
    top = grid::textGrob(
        "Sesgo en el número de días de riesgo de mildiu (ERA5-Land vs PTI)",
        gp = grid::gpar(fontsize = 16, fontface = "bold")
    )
)

dev.off()

pdf 
  2

## Corr ndays

In [20]:
calc_cor_pval_climate4R_blocked = function(model_data, obs_data, threshold = 0.05, block_size = 10) {
    ntime = dim(model_data$Data)[1]
    nlat  = dim(model_data$Data)[2]
    nlon  = dim(model_data$Data)[3]

    # Inicializar arrays finales
    cor_array  = matrix(NA, nrow = nlat, ncol = nlon)
    pval_array = matrix(NA, nrow = nlat, ncol = nlon)

    # Dividir latitudes en bloques
    blocks = split(seq_len(nlat), ceiling(seq_len(nlat)/block_size))

    # Procesar cada bloque
    for (b in seq_along(blocks)) {
        lat_idx = blocks[[b]]

        for (i in lat_idx) {
            for (j in seq_len(nlon)) {
                pred = model_data$Data[, i, j]
                obs  = obs_data$Data[, i, j]
                ok   = complete.cases(pred, obs)

                if (sum(ok) >= 10) {
                    r = cor(pred[ok], obs[ok])
                    n = sum(ok)
                    t = r * sqrt((n - 2) / (1 - r^2))
                    p = 2 * (1 - pt(abs(t), df = n - 2))

                    cor_array[i, j]  = r
                    pval_array[i, j] = p
                }
            }
        }

        # Liberar memoria del bloque
        rm(pred, obs, ok)
        gc()
    }

    # Construir grid de correlación
    cor_grid = list(
        Data     = cor_array,
        xyCoords = model_data$xyCoords,
        Variable = model_data$Variable
    )
    attr(cor_grid$Data, "dimensions") = c("lat", "lon")
    cor_grid$Dates = model_data$Dates
    class(cor_grid) = "grid"
    cor_grid$Variable$varName = "Corr"
    attr(cor_grid$Variable, "description") = "Mapa de correlaciones"
    attr(cor_grid$Variable, "units") = ""
    attr(cor_grid$Variable, "longname") = "Correlación"

    # Construir grid de p-valores
    pval_grid = list(
        Data     = pval_array,
        xyCoords = model_data$xyCoords,
        Variable = model_data$Variable
    )
    attr(pval_grid$Data, "dimensions") = c("lat", "lon")
    pval_grid$Dates = model_data$Dates
    class(pval_grid) = "grid"
    pval_grid$Variable$varName = "p-values"
    attr(pval_grid$Variable, "description") = "Mapa de p-valores"
    attr(pval_grid$Variable, "units") = ""
    attr(pval_grid$Variable, "longname") = "p-values"

    # Puntos significativos
    pts = map.stippling(
        climatology(pval_grid),
        threshold = threshold,
        condition = "LT",
        pch = 19,
        col = "black",
        cex = 0.05
    ) %>% suppressMessages() %>% suppressWarnings()

    return(list(cor = cor_grid, pval = pval_grid, pts = pts))
}

In [21]:
# Vector de meses en formato "01", "02", ..., "12"
meses = sprintf("%02d", 1:12)

# Lista para guardar resultados
resultados = lapply(meses, function(mes) {
    # Construir los nombres de los objetos para cada mes
    grid_era5 = get(paste0("grid_masked_era5_", mes))
    grid_pti   = get(paste0("grid_masked_pti_", mes))
    
    # Llamar a la función bloqueada
    calc_cor_pval_climate4R_blocked(grid_era5, grid_pti) %>% suppressWarnings()
})

# Asignar nombres a la lista para cada mes
names(resultados) = meses

# Guardar la lista completa en un archivo RDS
saveRDS(resultados, file = "corr_meses_mildiu_highres.rds")

In [23]:
cor = readRDS("../../../data/compound_study_index/high_resolution/corr_meses_oidio_highres.rds")

# Vector de meses
meses = sprintf("%02d", 1:12)

# Generar y guardar los plots
for (mes in meses) {
    assign(paste0("c_", mes),
           spatialPlot(
               climatology(cor[[mes]]$cor),
               backdrop.theme = "countries",
               main = paste("Corr ndays (Mes", mes, ")"),
               col.regions = color,
               sp.layout = list(cor[[mes]]$pts),
               at = seq(-1, 1, 0.1)) %>% suppressMessages() %>% suppressWarnings())
}

# Guardar en un PNG multipanel
png("corr_ndays_mildiu_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots
plots = mget(paste0("c_", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(
    grobs = plots,
    ncol = 4,
    top = grid::textGrob(
        "Correlación en el número de días de riesgo de mildiu (ERA5-Land vs PTI)",
        gp = grid::gpar(fontsize = 16, fontface = "bold")))

dev.off()

Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”
Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”
Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”
Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”
Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”
Warning message in min(x):
“ningún argumento finito para min; retornando Inf”
Warning message in max(x):
“ningun argumento finito para max; retornando -Inf”


pdf 
  2

## RMSE ndays

In [24]:
# Función para calcular RMSE entre dos grids climate4R
calc_rmse = function(pred, obs) {
    
    # Error = grid1 - grid2
    err = gridArithmetics(pred, obs, operator = "-")
    
    # Error^2
    err2 = gridArithmetics(err, err, operator = "*")

    # RMSE, promedio temporal
    rmse_field = sqrt(apply(err2$Data, c(2, 3), mean, na.rm = TRUE))

    # Reconstruir el objeto grid
    rmse_grid = list()
    rmse_grid$Data = rmse_field
    attr(rmse_grid$Data, "dimensions") = c("lat", "lon")
    rmse_grid$xyCoords = pred$xyCoords
    rmse_grid$Variable = pred$Variable
    rmse_grid$Dates = pred$Dates
    class(rmse_grid) = "grid"

    # Ajustar metadatos
    rmse_grid$Dates = err$Dates
    rmse_grid$Variable$varName = "RMSE"
    rmse_grid$Variable$longname = "Root Mean Square Error"
    rmse_grid$Variable$description = "RMSE"
    rmse_grid$Variable$units = "Número de días"
    
    return(rmse_grid)
}

In [31]:
for (i in 1:12) {
    mes = sprintf("%02d", i)
    
    # Calcular diferencias
    assign(paste0("rmse_", mes),
           calc_rmse(get(paste0("grid_masked_era5_", mes)),
                     get(paste0("grid_masked_pti_", mes))) %>% suppressMessages %>% suppressWarnings)
    
    # Graficar
    assign(paste0("rmse", mes),
           spatialPlot(get(paste0("rmse_", mes)),
                       backdrop.theme = "countries",
                       main = paste("RMSE ndays (Mes", mes, ")"),
                       col.regions = color,
                       at = seq(0, 7, 0.1)) %>% suppressMessages %>% suppressWarnings)
}

In [32]:
png("rmse_ndays_mildiu_vid.png", width = 2000, height = 1000, res = 150)

# Recojo todos los plots
plots = mget(paste0("rmse", sprintf("%02d", 1:12)))

# Organizo en grid
grid.arrange(
    grobs = plots,
    ncol = 4,
    top = grid::textGrob(
        "RMSE del número de días de riesgo de mildiu (ERA5-Land vs PTI)",
        gp = grid::gpar(fontsize = 16, fontface = "bold")
    )
)

dev.off()

pdf 
  2

## Ratio de varianzas

In [36]:
# Vector con los meses
meses = sprintf("%02d", 1:12)

# Lista para guardar los plots
plots = list()

for (m in meses) {
    # Nombres dinámicos
    era5_name = paste0("grid_masked_era5_", m)
    pti_name  = paste0("grid_masked_pti_", m)
    
    # Recupero objetos
    era5_obj = get(era5_name)
    pti_obj  = get(pti_name)
    
    # Calculo varianzas
    var_era5 = climatology(era5_obj,
                           clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages() %>% suppressWarnings()
    
    var_pti = climatology(pti_obj,
                          clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages() %>% suppressWarnings()
    
    # Cociente
    rv = gridArithmetics(var_era5, var_pti, operator = "/")
    
 # Si no hay datos válidos, grid vacío con mismo dominio
    if (all(is.na(rv$Data)) | all(rv$Data == 0, na.rm = TRUE) | all(!is.finite(rv$Data))) {
        rv_empty = rv
        rv_empty$Data[] = NA
        
        plots[[paste0("rv", m)]] = spatialPlot(
            rv_empty,
            backdrop.theme = "countries",
            col.regions = color,
            main = paste("Mes", m),
            at = seq(0, 4, 0.1))
    } else {
        plots[[paste0("rv", m)]] = spatialPlot(
            rv,
            backdrop.theme = "countries",
            col.regions = color,
            main = paste("Mes", m),
            at = seq(0, 4, 0.1))
    }
}

In [37]:
# Exportar en PNG
png("rv_ndays_mildiu_vid.png", width = 2000, height = 1000, res = 150)

grid.arrange(grobs = plots,
             ncol = 4,
             top = grid::textGrob(
                 "Ratio de varianzas del número de días de riesgo de mildiu (ERA5-Land vs PTI)",
                 gp = grid::gpar(fontsize = 16, fontface = "bold"))
)

dev.off()

pdf 
  2